In [250]:
import pandas as pd
from scipy.stats import poisson
import json

In [279]:
jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
with open(jagglar_rate_path, "r", encoding="utf-8") as f:
    model_data_dict = json.load(f)

# 設定推定関数
def estimate_setting_probs(row, model_data_dict, limit_games=5000):
    # JSONファイルの確率表を読み込む
    model = row["model_name"]
    games = row["game"]
    
    if games < limit_games or pd.isna(row["RB_rate"]) or pd.isna(row["Total_rate"]):
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    
    use_grape = not pd.isna(row["Grape_rate"]) and row["Grape_rate"] != 0
    
    rb_actual = row["RB"]
    total_actual = row["BB"] + row["RB"]
    if use_grape:
        grape_actual = round(games / row["Grape_rate"])
    

    model_data = model_data_dict.get(model)
    if not model_data:
        return {f"設定{i}": 0.0 for i in range(1, 7)}

    likelihoods = {}
    for setting, values in model_data.items():
        try:
            rb_rate = float(values["RB_RATE"].split("/")[1])
            total_rate = float(values["TOTAL_RATE"].split("/")[1])
            grape_rate = float(values["GRAPE_RATE"].split("/")[1])

            expected_rb = games / rb_rate
            expected_total = games / total_rate
            expected_grape = games / grape_rate
            
            rb_likelihood = poisson.pmf(rb_actual, expected_rb)
            total_likelihood = poisson.pmf(total_actual, expected_total)
            
            # Grapeを使うかどうか
            if use_grape:
                grape_rate = float(values["GRAPE_RATE"].split("/")[1])
                expected_grape = games / grape_rate
                grape_likelihood = poisson.pmf(grape_actual, expected_grape)
                likelihood = rb_likelihood * total_likelihood * grape_likelihood
            else:
                likelihood = rb_likelihood * total_likelihood

            likelihoods[int(setting)] = likelihood
        except:
            continue

    total_sum = sum(likelihoods.values())
    if total_sum == 0:
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    probs = {f"設定{s}": round(l / total_sum, 2) for s, l in likelihoods.items()}
    
    return probs

In [278]:
df = pd.read_csv('df_preprocessing.csv')
df_probs = df.apply(estimate_setting_probs, axis=1, result_type="expand")
df_probs["5_more"] = df_probs[["設定5", "設定6"]].sum(axis=1)

In [264]:
pd.concat([df.iloc[:, :12], df_probs["5_more"]], axis=1)

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,Grape_rate,5_more
100,EXA FIRST,2025-05-28,アイムジャグラーEX-TP,1102,1248,5,2,-50,249.6,624.0,178.3,5.93,0.00
101,EXA FIRST,2025-05-28,アイムジャグラーEX-TP,1103,1409,8,5,732,176.1,281.8,108.4,6.14,0.00
102,EXA FIRST,2025-05-28,アイムジャグラーEX-TP,1104,5448,32,9,2285,170.2,605.3,132.9,5.99,0.03
103,EXA FIRST,2025-05-28,アイムジャグラーEX-TP,1105,704,2,3,-150,352.0,234.7,140.8,6.58,0.00
104,EXA FIRST,2025-05-28,アイムジャグラーEX-TP,1106,969,5,2,132,193.8,484.5,138.4,6.71,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,EXA FIRST,2025-05-27,ジャグラーガールズ,1147,1371,4,3,-379,342.8,457.0,195.9,0.00,0.00
296,EXA FIRST,2025-05-27,ジャグラーガールズ,1148,4162,21,11,994,198.2,378.4,130.1,0.00,0.00
297,EXA FIRST,2025-05-27,ジャグラーガールズ,1149,830,1,1,-691,830.0,830.0,415.0,0.00,0.00
298,EXA FIRST,2025-05-27,ジャグラーガールズ,1150,1685,3,4,-994,561.7,421.2,240.7,0.00,0.00
